In [1]:
import guide_tree_group
import guide_tree_compute
import blastn_process
import MHG_partition
import process_mhg
import consensus_mhg
import copy

In [2]:
test_genome_dir = "/shared/yy70/github/MHG-II/test_dataset/far_genomes/"
temp_genome_dir = 'temp_far'
kmer_size = 16
thread = 1
mash_tree_path = "nj_far.newick"
blastn_dir = "temp_blastn_far"

In [3]:
# Concatenate fasta genomes to a directory; All assemblies are concat to the same file
accDic = guide_tree_compute.concat_fasta(test_genome_dir, temp_genome_dir)
# Mash to construct a distance matrix, and perform NJ. mash_tree_path contains the tree
guide_tree_compute.mash_distance_matrix_njtree(temp_genome_dir, mash_tree_path, kmer_size, thread)
# Reroot to obtain the lowest height reroot tree
rerooted_tree = guide_tree_group.shortest_reroot(mash_tree_path)
# Visited_node_MHG: internal node(key), MHG set(value); remaining_pair: list of remaining 
visited_node_MHG, remaining_pair = guide_tree_group.initial_taxa_internal(rerooted_tree)
# ready_MHG: None if next internal is NOT ready, otherwise key: internal node, value: MHG set

Sketching temp_far/cat.fa (provide sketch file made with "mash sketch" to skip)...done.


In [ ]:
while remaining_pair:
    next_internal_ready_boolean, internal_node_taxa, ready_MHG_dict, visited_node_MHG, remaining_pair = guide_tree_group.give_me_the_next_visit(visited_node_MHG, remaining_pair)
    if next_internal_ready_boolean:
        #这里现在不行 assume的是两个children都是leaf taxa；如果一个children是internal node，blastn_next要重新写
        blastn_out_path = blastn_process.blastn_next(ready_MHG_dict, blastn_dir, temp_genome_dir)
        df, check_dict = MHG_partition.parseBlastXML(blastn_out_path)
        df = MHG_partition.trim_fully_contain(df, check_dict)
        mhg_list = MHG_partition.mhg(df, 2)
        if '|' not in internal_node_taxa:
            # Case 1: two children nodes are both leaf nodes
            pan_mhg_list = process_mhg.pangenome(mhg_list, accDic)
        else:
            # Case 2: two children nodes have an internal node
            consensus_mhg_list = consensus_mhg.consensus_to_blocks(mhg_list, ready_MHG_dict)
            pan_mhg_list = process_mhg.pangenome(consensus_mhg_list, accDic)
        refName_refBlcok_dict, ref_mhg_dict = process_mhg.mafft_consensus_mhg(pan_mhg_list, accDic)
        merged_internal_name = internal_node_taxa.replace(",","|")
        visited_node_MHG[merged_internal_name] = [refName_refBlcok_dict, ref_mhg_dict]
        # Convert consensus ref alignments to sequences, write to a new fasta and be ready for next blastn
        process_mhg.ref_alignment_to_fasta(merged_internal_name, temp_genome_dir, refName_refBlcok_dict)
    else:
        print("Next internal node is not ready")

In [ ]:
for block_list in pan_mhg_list[50:100]:
    block_string_list = [b.block_string for b in block_list]
    print(len(block_string_list),"blocks in mhg")
    print(print_alignment('./temp_far/', block_string_list))
    print("")

In [51]:
df[df['alignmentLength'] > 100]

,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
0,"ref_0:G001277235,G000276705","ref_5061:G000412335,G000262305",80.844,569,109,0,1,569,19,587,1.908050e-131,465,TTGTGGGCCTTGCCAACCCCGGCGCGGAATACGCCGCTACCCGCCA...,TTGTCGGCCTGGCCAACCCGGGCGCTGAGTATGCCGCCACCCGCCA...
61,"ref_1:G001277235,G000276705","ref_3938:G000412335,G000262305",86.375,411,56,0,1,411,7,417,4.478580e-132,467,GAGGTAGCAAGTGACCATTGCTATTGTAATAGGCACACATGGTTGG...,GAGGTAGCAAGTGACTATTGCTATTGTCATAGGCACACATGGCTGG...
181,"ref_5:G001277235,G000276705","ref_5139:G000412335,G000262305",85.030,167,25,0,2,168,518,352,3.434130e-45,177,TCCCAGTCAGTCACTTCTTTCTCCGCTTGGTCTTTGGCGTAACCGT...,TCCCAGTCCTTCACTTCTTTCTCTGCCTGGTCTTTGGCATAGCCGT...
220,"ref_7:G001277235,G000276705","ref_3696:G000412335,G000262305",81.319,1638,303,3,5,1639,91,1728,0.000000e+00,1365,CGACGCCAGCATGCCGCGCAGGATGTTCAGCTCCTGGCTTTCCGGG...,CGACGCCAGAATGCCGCGCAAAATATTTAACTCCTCACTTTCCGGG...
370,"ref_10:G001277235,G000276705","ref_3241:G000412335,G000262305",80.389,566,106,5,2,567,1034,1594,2.392630e-123,439,ATGGCCTATAACCGTCTGAACCAGCGCGTGAACAAGCTGGAGCTGA...,ATGGCGTACAACCGCCTCAGCCAGCGGGTGAACAAACTGGAACTTA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829864,"ref_15640:G001277235,G000276705","ref_7295:G000412335,G000262305",73.554,121,32,0,35817,35937,4689,4809,1.128050e-03,48,GTGAAATCGCACACCTGGTGATTGATTCCACCGGGCTGAAGGTCTT...,GTGAAATCGTCCATTTGGTAATAGATTCAGCAGGGGTGAAAGTCGT...
830047,"ref_15646:G001277235,G000276705","ref_9814:G000412335,G000262305",80.977,573,107,2,1760,2330,3708,3136,3.042830e-130,464,CTCACGGACACCCATGAAAGCCGCCAGCTTATGGTTTTGTCGCTGG...,CTCACCGGCACCCACGAAAGCCGCCAGCTGATGGTACTGTCGCTGG...
830087,"ref_15647:G001277235,G000276705","ref_9814:G000412335,G000262305",77.677,775,173,0,179,953,2880,2106,2.597310e-139,492,CTGACCCGCCTCAGCCATAGCCAGTGCGAACTGCTGGCCAGCGATG...,CTGACACACATCAGCCACAGCCAGTGCGAGCTACAGGCCAGTGACG...
830215,"ref_15650:G001277235,G000276705","ref_2853:G000412335,G000262305",85.102,537,80,0,17875,18411,1,537,1.957820e-161,571,CCGTGTCTCATGTCGCGCGCCTGCATGGCATCAATGCAAACCAGAT...,CCGTGTCCCACGTTGCCCGTCTGCATGGCATAAATGCCAACCAGAT...


In [65]:
df[df['alignmentLength'] > 60]

,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
0,"ref_0:G001277235,G000276705","ref_5061:G000412335,G000262305",80.844,569,109,0,1,569,19,587,1.908050e-131,465,TTGTGGGCCTTGCCAACCCCGGCGCGGAATACGCCGCTACCCGCCA...,TTGTCGGCCTGGCCAACCCGGGCGCTGAGTATGCCGCCACCCGCCA...
61,"ref_1:G001277235,G000276705","ref_3938:G000412335,G000262305",86.375,411,56,0,1,411,7,417,4.478580e-132,467,GAGGTAGCAAGTGACCATTGCTATTGTAATAGGCACACATGGTTGG...,GAGGTAGCAAGTGACTATTGCTATTGTCATAGGCACACATGGCTGG...
181,"ref_5:G001277235,G000276705","ref_5139:G000412335,G000262305",85.030,167,25,0,2,168,518,352,3.434130e-45,177,TCCCAGTCAGTCACTTCTTTCTCCGCTTGGTCTTTGGCGTAACCGT...,TCCCAGTCCTTCACTTCTTTCTCTGCCTGGTCTTTGGCATAGCCGT...
220,"ref_7:G001277235,G000276705","ref_3696:G000412335,G000262305",81.319,1638,303,3,5,1639,91,1728,0.000000e+00,1365,CGACGCCAGCATGCCGCGCAGGATGTTCAGCTCCTGGCTTTCCGGG...,CGACGCCAGAATGCCGCGCAAAATATTTAACTCCTCACTTTCCGGG...
312,"ref_9:G001277235,G000276705","ref_4142:G000412335,G000262305",87.805,82,10,0,126,207,91,10,6.079940e-22,100,TATCGCCGTTGTTCAGCGCTTCGTTCGGCAGAACGTAGTCGTTAAA...,TATCGCCTTTGCTCAGCGCTTCGTTAGGCAGAACGTAATCGGTAAA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829887,"ref_15641:G001277235,G000276705","ref_2882:G000412335,G000262305",75.269,93,23,0,113,205,393,301,2.088570e-05,46,AAATATACACTAACGCCAAAGCTGTGGGATTATCTCTATAAGTACG...,AAATATATTCTGACGCCGACCCTGTGGCACTACCTCTACAAGTACG...
830047,"ref_15646:G001277235,G000276705","ref_9814:G000412335,G000262305",80.977,573,107,2,1760,2330,3708,3136,3.042830e-130,464,CTCACGGACACCCATGAAAGCCGCCAGCTTATGGTTTTGTCGCTGG...,CTCACCGGCACCCACGAAAGCCGCCAGCTGATGGTACTGTCGCTGG...
830087,"ref_15647:G001277235,G000276705","ref_9814:G000412335,G000262305",77.677,775,173,0,179,953,2880,2106,2.597310e-139,492,CTGACCCGCCTCAGCCATAGCCAGTGCGAACTGCTGGCCAGCGATG...,CTGACACACATCAGCCACAGCCAGTGCGAGCTACAGGCCAGTGACG...
830215,"ref_15650:G001277235,G000276705","ref_2853:G000412335,G000262305",85.102,537,80,0,17875,18411,1,537,1.957820e-161,571,CCGTGTCTCATGTCGCGCGCCTGCATGGCATCAATGCAAACCAGAT...,CCGTGTCCCACGTTGCCCGTCTGCATGGCATAAATGCCAACCAGAT...


In [55]:
df[df['alignmentLength'] > 20]

,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
0,"ref_0:G001277235,G000276705","ref_5061:G000412335,G000262305",80.844,569,109,0,1,569,19,587,1.908050e-131,465,TTGTGGGCCTTGCCAACCCCGGCGCGGAATACGCCGCTACCCGCCA...,TTGTCGGCCTGGCCAACCCGGGCGCTGAGTATGCCGCCACCCGCCA...
4,"ref_0:G001277235,G000276705","ref_11939:G000412335,G000262305",88.462,26,3,0,333,358,81,106,3.218180e-01,33,CTGAAAGACATCATCAGCAAGCTTGG,CTGAAAAACATCATCAGCATGATTGG
13,"ref_0:G001277235,G000276705","ref_5879:G000412335,G000262305",90.909,22,2,0,401,422,9208,9229,1.220100e+00,31,TCCGGGCGATAAAAATAAAGTT,TCCAGGCGATAAAAATTAAGTT
14,"ref_0:G001277235,G000276705","ref_5720:G000412335,G000262305",90.909,22,2,0,69,90,2438,2459,1.220100e+00,31,GATTTACTGGCTGATCGCCACC,GATTTACTGGCCGATAGCCACC
17,"ref_0:G001277235,G000276705","ref_190:G000412335,G000262305",88.000,25,3,0,387,411,173,197,1.220100e+00,31,CTGGGCATCGGCCATCCGGGCGATA,CTGGACAACGGCCATCCGGTCGATA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
830266,"ref_15650:G001277235,G000276705","ref_3094:G000412335,G000262305",84.848,33,5,0,21611,21643,1748,1716,5.933060e+00,35,TCCGTTCTCTGACGGAAGCCATCGACACCACCA,TCCGTGCTCTGACGGAAGAGACCGGCACCACCA
830267,"ref_15650:G001277235,G000276705","ref_2365:G000412335,G000262305",91.667,24,2,0,4955,4978,3059,3082,5.933060e+00,35,GGCGCTCAGACGCTGGCTGGTCAG,GGCGCTCAGCAGCTGGCTGGTCAG
830268,"ref_15650:G001277235,G000276705","ref_1217:G000412335,G000262305",95.238,21,1,0,2213,2233,203,183,5.933060e+00,35,ATCGGCCAGGCAATCAGCGCG,ATCTGCCAGGCAATCAGCGCG
830270,"ref_15650:G001277235,G000276705","ref_1039:G000412335,G000262305",91.667,24,2,0,3044,3067,566,589,5.933060e+00,35,CGCGGGCGGCAAGCGCCATCGCTT,CGCGGGCGCCCAGCGCCATCGCTT


In [ ]:
dict_1 = visited_node_MHG['G000412335,G000262305'][0]
dict_2 = visited_node_MHG['G000412335,G000262305'][1]
for ref_name in list(dict_1.keys())[:100]:
    ref_block = dict_1[ref_name]
    mhg = dict_2[ref_block].mhg_list
    print("ref")
    print(ref_block.alignment)
    for b in mhg:
        print(b.alignment)
    print("")